# This is a simple tcp server test :

In [ ]:
import socket
import threading
import time
import json, types,string

class ClientData():
    clientsocket=''
    
    
    def __init__(self,client):
        self.clientsocket = client
        

class SocketServer(socket.socket):
    server_name='Rem'
    clients = []
    message = "this is a message."
#     =====================
#     HERE IS SERVER INIT
#     =====================    
    def __init__(self):
        socket.socket.__init__(self)
        #To silence- address occupied!!
        self.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.bind(('0.0.0.0', 5566))
        self.listen(5)

    def run(self):
        print ("Server started")
        try:
            self.accept_clients()
        except Exception as ex:
            print (ex)
        finally:
            print ("Server closed")
            for client in self.clients:
                client.close()
            self.close()
    def decode_data(self,data):
        splited = data.decode("utf-8").split(',')
        return splited 
#     =====================
#     HERE IS TEST FUNCTION
#     =====================
    
    def sendTime(self, client,jdata):
        import locale
        locale.setlocale(locale.LC_CTYPE, 'chinese')
        dict_hour={0:'半夜12點',1:'半夜1點',2:'半夜2點',3:'半夜3點',4:'半夜4點',5:'凌晨5點',6:'凌晨6點',7:'早上7點',
        8:'早上8點',9:'早上9點',10:'早上10點',11:'中午11點',12:'中午12點',13:'中午1點',14:'下午2點',15:'下午3點',
        16:'下午4點',17:'下午5點',18:'晚上6點',19:'晚上7點',20:'晚上8點',21:'晚上9點',22:'晚上10點',23:'晚上11點',}
        year, month, day, hour, minute,second = time.strftime("%Y,%m,%d,%H,%M,%S").split(',')

        time_str="現在是"+dict_hour[int(hour)]+"%s分%s秒呦" %(minute,second,)
        msg = {'cmd':'say','name':self.server_name, 'data':time_str}
        jmsg = json.dumps(msg)
        client.send(str.encode(jmsg))
    
    def greed(self,client,jdata):
        msg = {'cmd':'greed','name':self.server_name, 'data':'wtf'}
        jmsg = json.dumps(msg)
        client.send(str.encode(jmsg))
    
    def someoneSay(self,client,jdata):
        client_name=jdata['name']
        client_msg=jdata['data']
        msg =  {'cmd':'say','name':client_name, 'data':client_msg}
        jmsg=json.dumps(msg)
        self.sendAll(jmsg)
        
    def sendAll(self,string):
        for client in self.clients:
            try :
                client.clientsocket.send(str.encode(string));
            except Exception as ex:
                print (ex)
                self.disconnect(client)
            finally:
                pass
            
        
        
#     WHEN CLIENT DISCONNECT
    def disconnect(self,client):
        #Removing client from clients list
        self.clients.remove(client)
        #Client Disconnected
        msg = {'cmd':'disconnect accept','name':self.server_name, 'data':'bye'}
        jmsg = json.dumps(msg)
        client.clientsocket.send(str.encode(jmsg))
        self.onclose(client)
        #Closing connection with client
        client.clientsocket.close()


    dictCmdFunc= {'what time':sendTime,
              'my name is':greed,
              'say':someoneSay}
    def recieve(self, client):
        while 1:
            data = client.clientsocket.recv(1024)
            #Message Received
            self.onmessage(client, data)
            jdata = json.loads(data)

            recv_cmd = jdata['cmd']
            
            if recv_cmd == 'disconnect':
                break
            else:                
                #Start thread 
                threading.Thread(target=self.dictCmdFunc[recv_cmd],args=(self,client.clientsocket,jdata,)).start()

        self.disconnect(client)
    
    def accept_clients(self):
        while 1:
            print("waiting for Client")
            (clientsocket, address) = self.accept()
            
            new_client = ClientData(clientsocket)
            #Adding client to clients list
            self.clients.append(new_client)
            #Client Connected
            self.onopen(clientsocket)
            #Start listening
            threading.Thread(target=self.recieve,args=(new_client,)).start()

    def broadcast(self, message):
        #Sending message to all clients
        for client in self.clients:
            client.clientsocket.send(message)
    # ===================
    # SOME EVENT FUNCTION
    # ===================
    def onopen(self, client):
        print ("Client Connected \n",client)

    def onmessage(self, client, message):
        print ("Client send message ",message)
        
    def onclose(self, client):
        print ("Client Disconnected")

def main():
    server = SocketServer()
    server.run()

if __name__ == "__main__":
    main()

Server started
waiting for Client
Client Connected 
 <socket.socket fd=1584, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 5566), raddr=('127.0.0.1', 59549)>
waiting for Client
Client send message  b'{"cmd": "say", "name": "Pohong", "data": "13\\n"}'
Client send message  b'{"cmd": "say", "name": "Pohong", "data": "1248\\n"}'
Client Connected 
 <socket.socket fd=1640, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 5566), raddr=('127.0.0.1', 59557)>
waiting for Client
Client send message  b'{"cmd": "say", "name": "Pohong", "data": "123456789\\n"}'
Client send message  b'{"cmd":"say","name":"pohong","data":"123456"}'
Client send message  b'{"cmd": "say", "name": "Pohong", "data": "77777\\n"}'
Client send message  b'{"cmd": "say", "name": "Pohong", "data": "6666666\\n"}'
Client send message  b'{"cmd": "say", "name": "Pohong", "data": "1\\n2\\n3\\n"}'
Client send message  b'{"cmd": "say", "name": "Pohong", "d

Exception in thread Thread-54:
Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\envs\test33\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\user\Anaconda3\envs\test33\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-d3babad1515c>", line 104, in recieve
    jdata = json.loads(data)
  File "C:\Users\user\Anaconda3\envs\test33\lib\json\__init__.py", line 354, in loads
    return _default_decoder.decode(s)
  File "C:\Users\user\Anaconda3\envs\test33\lib\json\decoder.py", line 339, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Users\user\Anaconda3\envs\test33\lib\json\decoder.py", line 357, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)



Client Connected 
 <socket.socket fd=1620, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 5566), raddr=('127.0.0.1', 59654)>
waiting for Client
Client send message  b'{"cmd": "say", "name": "Pohong", "data": "123132"}'
Client send message  b'{"cmd": "say", "name": "Pohong", "data": "hello \\u6709\\u4eba\\u5728\\u55ce?"}'
[WinError 10053] 連線已被您主機上的軟體中止。


Exception in thread Thread-58:
Traceback (most recent call last):
  File "<ipython-input-1-d3babad1515c>", line 74, in sendAll
    client.clientsocket.send(str.encode(string));
ConnectionAbortedError: [WinError 10053] 連線已被您主機上的軟體中止。

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\user\Anaconda3\envs\test33\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\user\Anaconda3\envs\test33\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-d3babad1515c>", line 69, in someoneSay
    self.sendAll(jmsg)
  File "<ipython-input-1-d3babad1515c>", line 77, in sendAll
    self.disconnect(client)
  File "<ipython-input-1-d3babad1515c>", line 90, in disconnect
    client.clientsocket.send(str.encode(jmsg))
ConnectionAbortedError: [WinError 10053] 連線已被您主機上的軟體中止。



Client Connected 
 <socket.socket fd=1736, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 5566), raddr=('127.0.0.1', 59662)>
waiting for Client
Client send message  b'{"cmd": "say", "name": "Pohong", "data": "123456"}'
Client send message  b'{"cmd": "say", "name": "Pohong", "data": "\\u54c8\\u56c9  \\u6709\\u4eba\\u5728\\u55ce????\\n"}'
Client send message  b'{"cmd": "say", "name": "Pohong", "data": "\\u4f60\\u597d\\u554a"}'
Client send message  b'{"cmd": "say", "name": "Pohong", "data": "\\u5b89\\u5b89\\u4f60\\u597d\\n"}'
Client send message  b'{"cmd": "say", "name": "Pohong", "data": "1\\n1\\n1\\n1\\n1\\n1\\n"}'
Client send message  b'{"cmd": "what time", "name": "", "data": ""}'
Client send message  b'{"cmd": "what time", "name": "", "data": ""}'
Client send message  b'{"cmd": "what time", "name": "", "data": ""}'
Client send message  b'{"cmd": "what time", "name": "", "data": ""}'
Client send message  b'{"cmd": "what time", "name": "", "data